In [1]:
import numpy as np
import polars as pl
import pandas as pd
import os
import random

In [ ]:
# Download the optimized data file into a folder. Use this code to create a folder of files. 
# The files are grouped by video, as each video has a number of frames associated with it. 

parquet_file = "emotion_landmark_dataset.parquet"
output_dir = "video_frames_npz"
os.makedirs(output_dir, exist_ok=True)

x_cols = [f"x_{i}" for i in range(478)]
y_cols = [f"y_{i}" for i in range(478)]
z_cols = [f"z_{i}" for i in range(478)]

chunk_size = 10000  # adjust to your memory

# Dictionary to accumulate frames per video
video_data = {}

lf = pl.scan_parquet(parquet_file)
total_rows = lf.select(pl.len()).collect()[0, 0]

def append_to_npz(filename, coords, emotions, frame_nums):
    if os.path.exists(filename):
        # Load old arrays and concatenate
        data = np.load(filename, allow_pickle=True)  # <-- allow_pickle=True
        old_coords = data['coords']
        old_emotions = data['emotions']
        old_frames = data['frame_nums']

        coords = np.concatenate([old_coords, coords], axis=0)
        emotions = np.concatenate([old_emotions, emotions], axis=0)
        frame_nums = np.concatenate([old_frames, frame_nums], axis=0)

    # Save updated arrays
    np.savez_compressed(filename, coords=coords, emotions=emotions, frame_nums=frame_nums)

for start in range(0, total_rows, chunk_size):
    df_chunk = lf.slice(start, chunk_size).collect().to_pandas()
    print(f"Processing rows {start}-{start + len(df_chunk)}")

    for video_name, group in df_chunk.groupby("video_filename", observed=False):
        x = group[x_cols].to_numpy(dtype='float32')
        y = group[y_cols].to_numpy(dtype='float32')
        z = group[z_cols].to_numpy(dtype='float32')
        coords = np.stack([x, y, z], axis=-1)
        emotions = group["emotion"].to_numpy()
        frame_nums = group["frame_num"].to_numpy()

        filename = os.path.join(output_dir, f"{video_name}.npz")
        append_to_npz(filename, coords, emotions, frame_nums)

print("All videos saved incrementally!")

Processing rows 0-10000
Processing rows 10000-20000
Processing rows 20000-30000
Processing rows 30000-40000
Processing rows 40000-50000
Processing rows 50000-60000
Processing rows 60000-70000
Processing rows 70000-80000
Processing rows 80000-90000
Processing rows 90000-100000
Processing rows 100000-110000
Processing rows 110000-120000
Processing rows 120000-130000
Processing rows 130000-140000
Processing rows 140000-150000
Processing rows 150000-160000
Processing rows 160000-170000
Processing rows 170000-180000
Processing rows 180000-190000
Processing rows 190000-200000
Processing rows 200000-210000
Processing rows 210000-220000
Processing rows 220000-230000
Processing rows 230000-240000
Processing rows 240000-250000
Processing rows 250000-260000
Processing rows 260000-270000
Processing rows 270000-280000
Processing rows 280000-290000
Processing rows 290000-300000
Processing rows 300000-310000
Processing rows 310000-320000
Processing rows 320000-330000
Processing rows 330000-340000
Pro

In [69]:
# Testing code. Good to use for implementation. 

folder = "video_frames_npz"

# Randomly pick a file
file_path = os.path.join(folder, random.choice(os.listdir(folder)))
print("Inspecting file:", file_path)

# Load the file
data = np.load(file_path, allow_pickle=True)

# Stored in a (num_frames, 478, 3) tensor. 
data_coords = data['coords']
num_frames = data_coords[:,0,0].size

# For a specific frame, grab the x, y and z coordinate data. 
frame_num = 0
x_coords = data_coords[frame_num, :, 0]
y_coords = data_coords[frame_num, :, 1]
z_coords = data_coords[frame_num, :, 2]

# For the specific frame, grab the emotion. 
frame_emotion = data['emotions'][frame_num]

Inspecting file: video_frames_npz/1079_TSI_SAD_XX.flv.npz
